In [ ]:
from ultralytics import YOLO
from utilities.text_extraction.detect_roi import *
model = YOLO(MODEL_PATH)

In [ ]:
img_path = r"C:\Users\Levi\Desktop\clock-roi-dataset-small\generalized_test_images\nba_1.png"
model(img_path, save = True)

# Image Preprocessing
***

In [ ]:
from PIL import Image

outpath = r"demos\example_images\preprocessed_images\pp.png"
img_path = r"demos\example_images\example_rois\example_roi_1.PNG"

image = Image.open(img_path)
image = np.array(image)
result = preprocess_image_test(image)

# cv2.imwrite(outpath, result)


# ROI Detection
***

In [ ]:
from video import Video
from utilities.text_extraction.detect_roi import detect_roi

video_path = r"demos\example_videos_DNC\clock_roi_eur_4.mp4"
video = Video(video_path)

detect_roi(video)


# Text Extraction
***

In [ ]:
import numpy as np
import cv2
from PIL import Image
from utilities.text_extraction.detect_roi import detect_roi
from utilities.text_extraction.timestamp_extraction import extract_timestamps_from_image
from utilities.text_extraction.preprocessing import preprocess_image

image = Image.open(r"demos/example_rois/example_roi_3.png")
arr = np.array(image)

out_path = "demos/output_images/pp.png"
cv2.imwrite(out_path, preprocess_image(arr))

results = extract_timestamps_from_image(arr, print_results=True)
print("quarter: ", results.quarter)
print("time_remaining: ", results.time_remaining)

## Easy-OCR is Just Better Lol?

In [ ]:
from easyocr import Reader
reader = Reader(['en'])

results = reader.readtext(arr)
print(results)


# Unit Testing
***

In [6]:
import os
from pickle import TRUE
import numpy as np
from PIL import Image
from utilities.text_extraction.timestamp_extraction import *


def test_ocr_benchmark():
    """Tests text-extraction on a set of benchmark images. Returns an accuracy score as a [0-1] float."""

    path_to_test_images = r"demos/example_rois"
    images = [path for path in os.listdir(path_to_test_images)]

    found, total = 0, 0
    for path in images:
        img_path = os.path.join(path_to_test_images, path)
        image = Image.open(img_path)
        img_arr = np.array(image)
        result = extract_timestamps_from_image(img_arr, print_results=False)
        if result.time_remaining and result.quarter:
            found += 1
        total += 1

    print(found / total)
    assert total > 0
    assert found == total

In [7]:
test_ocr_benchmark()

0.4


AssertionError: 